# Imports

In [ ]:
import pandas as pd
from pathlib import Path

# Functions

In [ ]:
def print_area(df, mask=None, text_mask=None):
    df = df.copy()
    if mask is None:
        pass
    else:
        df = df.loc[mask,:]
    print(f"Fläche {text_mask}: {df['flaeche'].sum():0.1f} ha")

# Parameters

In [ ]:
path_entw_system = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil\05 Berechnungen Grundlagen\8 Entwässerungskonzept\Grundlagen\Stats_AB-BG-EWD\System_Ist-geplant.csv")

In [ ]:
ewd_bz_2005 = {
    'Kernzone A': 56,
    'Kernzone B': 56,
    'Zweigeschossige Wohnzone': 42,
    'Dreigeschossige Wohnzone': 107,
    'Zweigeschossige Wohnzone mit Gewerbeerleichterung': 40,
    'Dreigeschossige Wohnzone mit Gewerbeerleichterung': 68,
    'Industriezone': 56,
    'Gewerbezone A': 20,
    'Gewerbezone B': 95,
    'Zone für öffentliche Bauten B': 90,
    'Strasse (Ausserorts)': 0,
    'Strasse (Innerorts)': 0,
    'Strassen (HLS)': 0,
    'Bahnareal': 0,
    'Freihaltezone kommunal': 0,
    'Kantonale Freihaltezone': 0,
    'Landwirtschaftszone kommunal': 0,
    'Kantonale Landwirtschaftszone': 0,
    'Erholungszone': 0,
    'Wald': 0
}

ewd_bz_ist = {
    'Kernzone A': 120,
    'Kernzone B': 110,
    'Industriezone': 56,
}

ew_ist = 17100
ew_plan = 25600

# Computations

In [ ]:
entw_system = pd.read_csv(path_entw_system, sep=';', encoding='ANSI')
entw_system['flaeche'] /= 1e4

## Check Areas

In [ ]:
# Fläche gesamt
print_area(entw_system, text_mask='gesamt')

# Fläche angeschlossen Ist
mask_ist = ~entw_system['knoten_sw_istref'].isna()
print_area(entw_system, mask=mask_ist, text_mask='Ist-angeschlossen')

# Fläche angeschlossen Planung
mask_plan = ~entw_system['knoten_sw_geplantref'].isna()
print_area(entw_system, mask=mask_plan, text_mask='Planung-angeschlossen')

# Damals überbaut: 499.0 ha

In [ ]:
# Ist - Nur angeschlossene
mask = ~entw_system['knoten_sw_istref'].isna()
system_ist = entw_system.loc[mask,:].copy(deep=True)

# Set EWD-2005
system_ist['einwohnerdichte_2005'] = 0.0
for bz, ewd in ewd_bz_2005.items():
    mask = system_ist['bezeichnung']==bz
    system_ist.loc[mask, 'einwohnerdichte_2005'] = ewd

# Set EWD-Ist
system_ist['einwohnerdichte_ist'] = system_ist['einwohnerdichte_2005']
for bz, ewd in ewd_bz_ist.items():
    mask = system_ist['bezeichnung']==bz
    system_ist.loc[mask, 'einwohnerdichte_ist'] = ewd

# Compute EW 2005 / Ist
system_ist['ew_2005'] = system_ist['flaeche'] * system_ist['einwohnerdichte_2005']
system_ist['ew_ist'] = system_ist['flaeche'] * system_ist['einwohnerdichte_ist']

# Compute new EWD-Ist
bz_ewd_fix = list(ewd_bz_ist.keys())
mask = system_ist['bezeichnung'].isin(bz_ewd_fix)
f_ist = (ew_ist - system_ist.loc[mask, 'ew_ist'].sum()) / (system_ist.loc[~mask, 'ew_ist'].sum())
system_ist.loc[~mask, 'einwohnerdichte_ist'] *= f_ist # NaN * f_ist = NaN

# Compute new EW-Ist
system_ist['ew_ist'] = system_ist['flaeche'] * system_ist['einwohnerdichte_ist']

In [ ]:
# Create new dictionary with EWD-Planung (s. ewd_bz_2005)
ewd_bz_plan = system_ist.loc[:,['bezeichnung', 'einwohnerdichte_ist']].groupby('bezeichnung').mean() # NaN not included in mean
ewd_bz_plan.fillna(0, inplace=True) # Landwirtschaftszone kommunal had mean = NaN
ewd_bz_plan = ewd_bz_plan.to_dict()['einwohnerdichte_ist']

In [ ]:
# Planung - Nur angeschlossene
mask = ~entw_system['knoten_sw_geplantref'].isna()
system_plan = entw_system.loc[mask,:].copy(deep=True)

# Set EWD-Planung
for bz, ewd in ewd_bz_plan.items():
    mask = system_plan['bezeichnung']==bz
    system_plan.loc[mask, 'einwohnerdichte_planung'] = ewd

# Compute new EWD-Planung
f_plan = ew_plan / (system_plan['einwohnerdichte_planung'] * system_plan['flaeche']).sum()
system_plan['einwohnerdichte_planung'] *= f_plan

# Compute EW-Planung
system_plan['ew_planung'] = system_plan['flaeche'] * system_plan['einwohnerdichte_planung']

In [ ]:
# Summary per Bauzone
res_ewd = pd.DataFrame(index=sorted(entw_system['bezeichnung'].unique()))
res_ewd['flaeche'] = entw_system[['bezeichnung', 'flaeche']].groupby('bezeichnung').sum()
res_ewd['flaeche_sw_ist'] = system_ist[['bezeichnung', 'flaeche']].groupby('bezeichnung').sum()
res_ewd['flaeche_sw_planung'] = system_plan[['bezeichnung', 'flaeche']].groupby('bezeichnung').sum()
res_ewd['einwohnerdichte_2005'] = system_ist[['bezeichnung', 'einwohnerdichte_2005']].groupby('bezeichnung').mean()
res_ewd['ew_2005'] = system_ist[['bezeichnung', 'ew_2005']].groupby('bezeichnung').sum()
res_ewd['einwohnerdichte_ist'] = system_ist[['bezeichnung', 'einwohnerdichte_ist']].groupby('bezeichnung').mean()
res_ewd['ew_ist'] = system_ist[['bezeichnung', 'ew_ist']].groupby('bezeichnung').sum()
res_ewd['einwohnerdichte_planung'] = system_plan[['bezeichnung', 'einwohnerdichte_planung']].groupby('bezeichnung').mean()
res_ewd['ew_planung'] = system_plan[['bezeichnung', 'ew_planung']].groupby('bezeichnung').sum()

# Reorder results
order_ewd = [14, 4, 6, 19, 15, 8, 9, 7, 16, 18, 1, 2, 17, 11, 12, 13, 20, 10, 3, 5]
res_ewd['order'] = order_ewd
res_ewd.sort_values(by='order', inplace=True)

# Compute sum / mean
sum_idx = 'SUM / MEAN'
res_ewd.loc[sum_idx, 'flaeche'] = res_ewd['flaeche'].sum()
res_ewd.loc[sum_idx, 'flaeche_sw_ist'] = res_ewd['flaeche_sw_ist'].sum()
res_ewd.loc[sum_idx, 'flaeche_sw_planung'] = res_ewd['flaeche_sw_planung'].sum()
res_ewd.loc[sum_idx, 'ew_2005'] = res_ewd['ew_2005'].sum()
res_ewd.loc[sum_idx, 'ew_ist'] = res_ewd['ew_ist'].sum()
res_ewd.loc[sum_idx, 'ew_planung'] = res_ewd['ew_planung'].sum()

# Save results
save_path = path_entw_system.parent / 'EWD_2005-Ist-Planung.xlsx'
res_ewd.to_excel(save_path)